# DEX 구성하기 [3] 유동성에 공급하기


---

* **DATE** : 2021.11.06
* **AUTHOR** : Kang Sang Jae
* **Reference** : 
    * [Taking undercollateralized loans for fun and for profit](https://samczsun.com/taking-undercollateralized-loans-for-fun-and-for-profit/)
    * [Arxiv Paper](https://arxiv.org/pdf/2103.12732.pdf)
    * [zuniswap with Clone Coding](https://github.com/Jeiwan/zuniswap)
    * [Uniswap V1 Code](https://github.com/Uniswap/v1-contracts)

In [2]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

## LP들의 행동, Provision & Withdrawal

![](https://imgur.com/ygzyb9G.png)

초기에 부은 유동성 풀은 정적인 상태인 것은 아니다. 이후에도 추가적으로 LP는 Pool에 추가적으로 납입할 수 있으며, 풀에서 자산을 인출으로써 자신의 손익을 확정지을 수도 있다.

우리의 Liquidity Pool을 관리하는 스마트 컨트랙트를 `Moniswap`이라 해보자. Moniswap의 Pool은 모두가 참여할 수 있는 형태로 관리한다고 해보자. 

그러면 우리는 Pool에 넣을 때, 이 Pool에 대한 각자의 지분 계산이 필수적이다. 이 지분을 보통 LP-Tokens라고 부른다. 

우선 계산을 편하게 하기 위해서 가상의 계정을 아래와 같이 구성해보자. 

In [1]:
class Accounts:
    """EOA (External Owned Acounts) 계정 정보
    """
    address = ""
    balance_X = 0 # x토큰 잔액
    balance_Y = 0 # y토큰 잔액
    
    def __init__(self, address, balance_X, balance_Y):
        self.address = address
        self.balance_X = balance_X
        self.balance_Y = balance_Y
        
    def calculate_TVL(self, price_X, price_Y):
        return self.balance_X * price_X + self.balance_Y * price_Y

현재 시장 내 플레이어들의 계좌 상태는 아래와 같다고 가정해보자.

In [3]:
master_account = Accounts("master",100_000, 200_000)

A_account = Accounts("A", 10_000, 20_000)
B_account = Accounts("B", 30_000, 15_000)
C_account = Accounts("C", 20_000, 40_000)
D_account = Accounts("D", 15_000, 25_000)

아래의 스마트 컨트랙트에 `master_account`가 초기에 자신의 전 자산을 다 납입했다고 해보자. 

In [5]:
class Moniswap:
    # M3O1의 이니셜을 따서 Moniswap이라 해보자
    fee = 0.003 # FEE는 0.3%만큼 뗀다고 해보자 
    lp_token_balances: dict # key는 address, value는 address
    
    # 토큰이 실제로 저장된 컨트랙트
    def __init__(self, account:Accounts, supply_X, supply_Y):
        assert account.balance_X >= supply_X, "X토큰이 공급물량보다 모자랍니다"
        assert account.balance_Y >= supply_Y, "Y토큰이 공급물량보다 모자랍니다"        
        
        # X의 초기 통화 공급량 : supply_X
        # Y의 초기 통화 공급량 : supply_Y
        self.supply_X = supply_X
        self.supply_Y = supply_Y
        
        # 공급한만큼 차감
        account.balance_X -= supply_X 
        account.balance_Y -= supply_Y
        
        # LP 토큰의 발행량은 X의 공급량와 동일하게 세팅
        self.lp_token_balances = {account.address : supply_X}
        
        
    def get_LP_balance(self, address):
        # 해당 주소에 할당되어 있는 LP Token
        return self.lp_token_balances.get(address, 0)
    
    def get_total_balance(self):
        # 전체 유동성 토큰 갯수
        return sum(self.lp_token_balances.values())
        
    @property
    def constant(self):
        # 추가적으로 풀에 토큰이 공급되거나, 빼지 않는 경우 
        # 해당 값은 계속 유지되어야 함
        return self.supply_X * self.supply_Y
        
    def calculate_TVL(self, price_X, price_Y):
        # 현재 price_X와 price_Y를 기준으로 풀의 총 가치 산정
        return self.supply_X * price_X + self.supply_Y * price_Y
        
    def swapX2Y(self, delta_X):
        # delta_X만큼의 X 토큰을 delta_Y만큼의 Y 토큰으로 교환
        # Fee를 뗀 만큼 유저에게 돌려준다
        delta_Y = (1- self.fee) * self.supply_Y * delta_X / (self.supply_X + delta_X)
        
        self.supply_X += delta_X
        self.supply_Y -= delta_Y
        return delta_Y        
        
    def swapY2X(self, delta_Y):
        # delta_Y만큼의 Y 토큰을 delta_X만큼의 X토큰으로 교환
        # Fee를 뗀 만큼 유저에게 돌려준다
        delta_X = (1- self.fee) * self.supply_X * delta_Y / (self.supply_Y + delta_Y)
        self.supply_X -= delta_X
        self.supply_Y += delta_Y
        return delta_X                

In [6]:
supply_X = 100_000
supply_Y = 200_000

moniswap = Moniswap(master_account, supply_X, supply_Y)

이렇게 구성할 경우 `master_account`로 할당되어 있는 LP Tokens의 양은 아래와 같다. 

In [7]:
moniswap.get_LP_balance(master_account.address)

100000

그리고 이제 master_acccount에 물려있는 자산은 모두 0원일 것이다.

In [8]:
print("balance X : ", master_account.balance_X)
print("balance Y : ", master_account.balance_Y)
print("P_x:20, P_y:10인 경우 : ", master_account.calculate_TVL(20,10))

balance X :  0
balance Y :  0
P_x:20, P_y:10인 경우 :  0


## 유동성 풀에 유동성을 추가하기

![](https://imgur.com/HvF9ipn.png)

너무나 당연한 얘기지만, **기존에 넣은 풀에 넣은 자산의 가치를 훼손하면 않으면서**, 풀에 유동성을 공급해야 한다. 그렇기 때문에 유동성을 주입할 때에는 SWAP의 비율을 해치면 안된다. 

AMM-Based DEX에서는 두 가지 일반 규칙이 있다.

````
1) The price of assets in an AMM pool stays constant for pure liquidity provision and withdrawal activities.

2) The invariant of an AMM pool stays constant for pure swapping activities.
````

1번은 LP의 행동(납입/출금)에 대한 규칙이고, 2번은 Trader의 행동에 대한 규칙이라고 생각하면 된다. 

이렇기 때문에, 유동성 풀에 납입하기 위해서는 당연히 두 토큰이 현재 시스템의 비율하고 동일하게 납입해야 한다. 보통의 경우엔, 유동성 풀에 납입 후 남는 만큼은 반환받는다.

In [14]:
class Moniswap:
    # M3O1의 이니셜을 따서 Moniswap이라 해보자
    fee = 0.003 # FEE는 0.3%만큼 뗀다고 해보자 
    lp_token_balances:dict # key는 address, value는 address
    
    # 토큰이 실제로 저장된 컨트랙트
    def __init__(self, account:Accounts, supply_X, supply_Y):
        assert account.balance_X >= supply_X, "X토큰이 공급물량보다 모자랍니다"
        assert account.balance_Y >= supply_Y, "Y토큰이 공급물량보다 모자랍니다"        
        
        # X의 초기 통화 공급량 : supply_X
        # Y의 초기 통화 공급량 : supply_Y
        self.supply_X = supply_X
        self.supply_Y = supply_Y
        
        # 공급한만큼 차감
        account.balance_X -= supply_X 
        account.balance_Y -= supply_Y
        
        # LP 토큰의 발행량은 X의 공급량와 동일하게 세팅
        self.lp_token_balances = {account.address:supply_X}
        
        
    def get_LP_balance(self, address):
        # 해당 주소에 할당되어 있는 LP Token
        return self.lp_token_balances.get(address, 0)
    
    def get_total_balance(self):
        # 전체 유동성 토큰 갯수
        return sum(self.lp_token_balances.values())
    
    def add_liquidity(self, account:Accounts, add_X, add_Y):
        """유동성 공급
        """
        required_Y = add_X * self.supply_Y/self.supply_X
        assert account.balance_X >= add_X, "X토큰이 공급물량보다 모자랍니다"
        assert account.balance_Y >= required_Y, "Y토큰이 공급물량보다 모자랍니다"
        
        # 찍어내는 토큰 양 계산
        # add_x / self.supply_X => 내가 % 기여했냐
        # 현재 총 LP token 양 * 기여분 => 내가 받을 토큰 양
        minted_amount = add_X / self.supply_X *  self.get_total_balance()
        
        # 유동성 풀에 토큰 넣기
        self.supply_X += add_X
        self.supply_Y += required_Y
        
        # 지갑에서 돈빼기
        account.balance_X -= add_X
        account.balance_Y -= required_Y
        
        # LP 토큰 넣어주기
        self.lp_token_balances[account.address] = (
            minted_amount + self.lp_token_balances.get(account.address, 0)
        )
        return minted_amount
        
    @property
    def constant(self):
        # 추가적으로 풀에 토큰이 공급되거나, 빼지 않는 경우 
        # 해당 값은 계속 유지되어야 함
        return self.supply_X * self.supply_Y
        
    def calculate_TVL(self, price_X, price_Y):
        # 현재 price_X와 price_Y를 기준으로 풀의 총 가치 산정
        return self.supply_X * price_X + self.supply_Y * price_Y
        
    def swapX2Y(self, delta_X):
        # delta_X만큼의 X 토큰을 delta_Y만큼의 Y 토큰으로 교환
        # Fee를 뗀 만큼 유저에게 돌려준다
        delta_Y = (1- self.fee) * self.supply_Y * delta_X / (self.supply_X + delta_X)
        
        self.supply_X += delta_X
        self.supply_Y -= delta_Y
        return delta_Y        
        
    def swapY2X(self, delta_Y):
        # delta_Y만큼의 Y 토큰을 delta_X만큼의 X토큰으로 교환
        # Fee를 뗀 만큼 유저에게 돌려준다
        delta_X = (1- self.fee) * self.supply_X * delta_Y / (self.supply_Y + delta_Y)
        self.supply_X -= delta_X
        self.supply_Y += delta_Y
        return delta_X                

In [31]:
master_account = Accounts("master",100_000, 200_000)
A_account = Accounts("A",100_000, 200_000)
B_account = Accounts("B",100_000, 200_000)

moniswap = Moniswap(master_account, supply_X, supply_Y)

In [32]:
moniswap.lp_token_balances

{'master': 100000}

In [33]:
moniswap.calculate_TVL(20,10)

4000000

In [34]:
moniswap.add_liquidity(A_account, 100_000, 200_000)

100000.0

In [35]:
moniswap.lp_token_balances

{'master': 100000, 'A': 100000.0}

In [36]:
moniswap.calculate_TVL(20,10)

8000000.0

In [37]:
B_account = Accounts("B",100_000, 200_000)

moniswap.add_liquidity(B_account, 100_000, 200_000)

100000.0

In [38]:
moniswap.calculate_TVL(20,10)

12000000.0

In [39]:
moniswap.lp_token_balances

{'master': 100000, 'A': 100000.0, 'B': 100000.0}

In [26]:
master_account = Accounts("master",100_000, 200_000)

moniswap = Moniswap(master_account, supply_X, supply_Y)

moniswap에 A 계좌가 참여한다고 해보자, 이때 X 토큰 5000개와 Y 토큰 10000개를 납입하면된다.

In [40]:
A_account = Accounts("A", 10_000, 20_000)

moniswap.add_liquidity(A_account, 5_000, 10_000)

5000.0

이러면 현재 LP 토큰의 비율은 아래와 같이 된다. 

In [41]:
moniswap.lp_token_balances

{'master': 100000, 'A': 105000.0, 'B': 100000.0}

그리고 남은 A 계정의 잔액은 아래와 같아진다.

In [42]:
print("balance X : ", A_account.balance_X)
print("balance Y : ", A_account.balance_Y)

balance X :  5000
balance Y :  10000.0


그럼 스왑이 발생하면 어떻게 될까? 토큰 X에 대해 1000개만큼의 스왑요청이 들어갔다고 해보자.

In [43]:
moniswap.swapX2Y(1000)

1987.483660130719

스왑 요청이 발생했으므로, X의 보유량이 늘고 Y의 보유량이 떨어질 것이다.

그와 무관하게 LP 토큰의 양은 아래와 같이 동일하게 구성된다. 

In [44]:
moniswap.lp_token_balances

{'master': 100000, 'A': 105000.0, 'B': 100000.0}

그럼 이렇게 스왑이 발생했을 때에 추가 납입을 시도하면 어떻게 될까? 

In [45]:
print("supply X : ", moniswap.supply_X)
print("supply Y : ", moniswap.supply_Y)

supply X :  306000
supply Y :  608012.5163398692


In [46]:
208024/106000

1.9624905660377359

지금의 swap ratio는 2.0보다는 약간 줄은 1.96 수준으로 되어 있다. 그럼 다시 원칙을 상기해보자.

````
1) The price of assets in an AMM pool stays constant for pure liquidity provision and withdrawal activities.
````

이때 각 토큰의 가치는 보존되어야 하기 때문에, swap 비율은 유지한채로 넣어주어야 한다.

In [47]:
moniswap.add_liquidity(A_account, 1000, 2000)

996.7320261437909

실제로 아까와 다르게 1000을 넣었더니 X의 가치가 좀 더 떨어졌기 때문에 990이라는 좀 더 낮은 수준의 LP 토큰을 받았다. 

In [48]:
A_account.balance_X

4000

In [49]:
A_account.balance_Y

8013.030992353369

In [50]:
moniswap.lp_token_balances

{'master': 100000, 'A': 105996.73202614379, 'B': 100000.0}

## 유동성 풀에서 출금하기 

![](https://imgur.com/61j7ynW.png)

유동성 풀에서 출금하는 것은 매우 간단하다. 전체 발행된 토큰 중 비율에 맞춰서 각 토큰 풀에서 취득하면 된다. 

In [51]:
class Moniswap:
    # M3O1의 이니셜을 따서 Moniswap이라 해보자
    fee = 0.003 # FEE는 0.3%만큼 뗀다고 해보자 
    lp_token_balances:dict # key는 address, value는 address
    
    # 토큰이 실제로 저장된 컨트랙트
    def __init__(self, account:Accounts, supply_X, supply_Y):
        assert account.balance_X >= supply_X, "X토큰이 공급물량보다 모자랍니다"
        assert account.balance_Y >= supply_Y, "Y토큰이 공급물량보다 모자랍니다"        
        
        # X의 초기 통화 공급량 : supply_X
        # Y의 초기 통화 공급량 : supply_Y
        self.supply_X = supply_X
        self.supply_Y = supply_Y
        
        # 공급한만큼 차감
        account.balance_X -= supply_X 
        account.balance_Y -= supply_Y
        
        # LP 토큰의 발행량은 X의 공급량와 동일하게 세팅
        self.lp_token_balances = {account.address:supply_X}
        
        
    def get_LP_balance(self, address):
        # 해당 주소에 할당되어 있는 LP Token
        return self.lp_token_balances.get(address, 0)
    
    def get_total_balance(self):
        # 전체 유동성 토큰 갯수
        return sum(self.lp_token_balances.values())
    
    def add_liquidity(self, account:Accounts, add_X, add_Y):
        """유동성 공급
        """
        required_Y = add_X * self.supply_Y/self.supply_X
        assert account.balance_X >= add_X, "X토큰이 공급물량보다 모자랍니다"
        assert account.balance_Y >= required_Y, "Y토큰이 공급물량보다 모자랍니다"
        
        # 찍어내는 토큰 양 계산
        minted_amount = add_X / self.supply_X * self.get_total_balance()
        
        # 유동성 풀에 토큰 넣기
        self.supply_X += add_X
        self.supply_Y += required_Y
        
        # 지갑에서 돈빼기
        account.balance_X -= add_X
        account.balance_Y -= required_Y
        
        # LP 토큰 넣어주기
        self.lp_token_balances[account.address] = (
            minted_amount + self.lp_token_balances.get(account.address, 0)
        )
        return minted_amount
    
    def remove_liquidity(self, account:Accounts, tokens):
        assert tokens >= self.lp_token_balances.get(account.address, 0)
        
        # 출금 금액 계산 하기
        ratio = tokens/self.get_total_balance()        
        out_X = ratio * self.supply_X
        out_Y = ratio * self.supply_Y
        
        # 풀에서 돈 빼기
        self.supply_X -= out_X
        self.supply_Y -= out_Y
        
        # 지갑에 돈 넣기
        account.balance_X += out_X
        account.balance_Y += out_Y
        
        # LP 토큰 제거하기
        self.lp_token_balances[account.address] -= tokens
        
        return out_X, out_Y
        
    @property
    def constant(self):
        # 추가적으로 풀에 토큰이 공급되거나, 빼지 않는 경우 
        # 해당 값은 계속 유지되어야 함
        return self.supply_X * self.supply_Y
        
    def calculate_TVL(self, price_X, price_Y):
        # 현재 price_X와 price_Y를 기준으로 풀의 총 가치 산정
        return self.supply_X * price_X + self.supply_Y * price_Y
        
    def swapX2Y(self, delta_X):
        # delta_X만큼의 X 토큰을 delta_Y만큼의 Y 토큰으로 교환
        # Fee를 뗀 만큼 유저에게 돌려준다
        delta_Y = (1- self.fee) * self.supply_Y * delta_X / (self.supply_X + delta_X)
        
        self.supply_X += delta_X
        self.supply_Y -= delta_Y
        return delta_Y        
        
    def swapY2X(self, delta_Y):
        # delta_Y만큼의 Y 토큰을 delta_X만큼의 X토큰으로 교환
        # Fee를 뗀 만큼 유저에게 돌려준다
        delta_X = (1- self.fee) * self.supply_X * delta_Y / (self.supply_Y + delta_Y)
        self.supply_X -= delta_X
        self.supply_Y += delta_Y
        return delta_X                

그럼 항상 

In [52]:
master_account = Accounts("master",100_000, 200_000)
A_account = Accounts("A", 10_000, 20_000)
print("A_account의 총 가치 : ", A_account.calculate_TVL(20, 10))

moniswap = Moniswap(master_account, supply_X, supply_Y)

# IN -> 
moniswap.add_liquidity(A_account, 10_000, 20_000)

## SWAP 발생 
moniswap.swapX2Y(1000)
moniswap.swapY2X(500)
moniswap.swapY2X(1200)
moniswap.swapY2X(500)
moniswap.swapY2X(1200)
moniswap.swapY2X(500)
moniswap.swapY2X(1200)
moniswap.swapY2X(500)
moniswap.swapY2X(1200)
moniswap.swapY2X(500)
moniswap.swapY2X(1200)
moniswap.swapX2Y(800)
moniswap.swapY2X(1000)
moniswap.swapX2Y(500)
moniswap.swapX2Y(1200)
moniswap.swapY2X(800)
moniswap.swapX2Y(1000)
moniswap.swapY2X(500)
moniswap.swapY2X(1200)
moniswap.swapX2Y(800)
moniswap.swapY2X(1000)
moniswap.swapX2Y(500)
moniswap.swapX2Y(1200)
moniswap.swapY2X(800)
moniswap.swapX2Y(1000)
moniswap.swapY2X(500)
moniswap.swapY2X(1200)
moniswap.swapX2Y(800)
moniswap.swapY2X(1000)
moniswap.swapX2Y(500)
moniswap.swapX2Y(1200)
moniswap.swapY2X(800)

# OUT -> 
moniswap.remove_liquidity(A_account, 10000)

# 나올 떄 시장에서 동일가치를 유지하였다면, 뺄 때 항상 차익이 발생한다.
print("A_account의 총 가치 : ", A_account.calculate_TVL(20,10))

A_account의 총 가치 :  400000
A_account의 총 가치 :  400166.23801687005
